## **Step 1 - keywords Extraction**
***

We have two datasets, one with dream text descriptions:

In [ ]:
from keyword_extractor import read_datasets, extract_and_save_keywords_from_dataframes
from yaml_parser import load_config
config = load_config()
dream_df, keywords_df = read_datasets(config)
dream_df.head()

And another one with interpretations of dreams according to keywords:

In [ ]:
keywords_df

Now, we will use pretrained LLMs in order to extract the given keywords from the keywords dataset , from the dream text description from the dream text dataset.

### **GPT2**
***

In [ ]:
dream_df = extract_and_save_keywords_from_dataframes()

## Step 2 - Summarize interpretations

### Load data and prepare (small) dataset for experimenting

In [ ]:
import pandas as pd
import pandasql as ps
from plotly import express as px
from datetime import datetime
from transformers import pipeline
from utils import  release_all_gpu_memory, save_df_as_pretty_html
from summarizer import load_causal_model, batch_generate_interpretations
import torch


In [ ]:
#dataset.to_csv("datasets/prepared_dataset.csv", index=False)
dataset = pd.read_csv("datasets/prepared_dataset.csv")

In [ ]:

dataset.sort_values("input_tokens_len", ascending=False, inplace=True)

In [ ]:
# filter dataset only to inputs of size that we can handle
dataset = dataset[dataset["input_tokens_len"] <1000]

In [ ]:
dataset

### Summarize with flan-T5-large model

In [ ]:
release_all_gpu_memory()

In [ ]:
# Step 1: Load FLAN-T5 model and tokenizer
model_name = "google/flan-t5-large"
model_name_short = model_name.split("/")[-1]
device = 0 if torch.cuda.is_available() else -1
model, tokenizer = load_causal_model(model_name)

In [ ]:
text2text_generator = pipeline(
        "text2text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=1024,           # ✅ allow longer input
        truncation=True,           # ✅ ensure truncation at tokenizer level
        device=device,
    )

In [ ]:
tstp = datetime.now().strftime(r"%y.%m.%d-%H")
result_df = batch_generate_interpretations(dataset, text2text_generator, batch_size=100, max_length=250)


In [ ]:
postproc = lambda out: out["generated_text"].strip()
result_df["interpretation"] = result_df["interpretation"].apply(postproc)


In [ ]:
result_df

In [ ]:
result_df.columns

In [ ]:
save_df = result_df[['prompt', 'symbols','dream', 'interpretation']]

path = f"output/{model_name_short}_{tstp}"

save_df_as_pretty_html(save_df, path + ".html")

save_df.to_csv(path + ".csv", index = False)

In [ ]:
result_df.interpretation.str.len()

### Summarize with Mistral model

In [ ]:
from summarizer import load_mistral_4bit_model, find_max_batch_size
print("Loading Mistral-7B-Instruct in 4-bit...")

model_name = "mistralai/Mistral-7B-Instruct-v0.2"
model_name_short = model_name.split("/")[-1]

model_family="decoder"

# max_new_tokens=256         max_new_tokens=max_new_tokens,


#### measure maximum batch size a given machine can hadle without OOMing


In [ ]:
sample_prompt = dataset.input[0]
print(f"Sample prompt length (characters): {len(sample_prompt)}")
model, tokenizer = load_mistral_4bit_model(model_name)
optimal_batch_size = find_max_batch_size(model, tokenizer, sample_prompt, max_possible=2048, max_length=1024) # TODO: measure with max_new_tokens

first run on 24 Gb GPU: 
```
Sample prompt length (characters): 2437  
Trying batch_size = 512... ❌ OOM  
Trying batch_size = 256... ❌ OOM  
Trying batch_size = 128... ❌ OOM  
Trying batch_size = 64... ❌ OOM  
Trying batch_size = 32... ✅ success  
Trying batch_size = 48... ✅ success  
Trying batch_size = 56... ✅ success  
Trying batch_size = 60... ❌ OOM  
Trying batch_size = 58... ❌ OOM  
Trying batch_size = 57... ✅ success  

✅ Optimal batch size: 57
```

second run on 24 Gb GPU: 
```
Sample prompt length (characters): 2437
Trying batch_size = 1024... ❌ OOM
Trying batch_size = 512... ❌ OOM
Trying batch_size = 256... ❌ OOM
Trying batch_size = 128... ❌ OOM
Trying batch_size = 64... ✅ success
Trying batch_size = 96... ✅ success
Trying batch_size = 112... ❌ OOM
Trying batch_size = 104... ❌ OOM
Trying batch_size = 100... ❌ OOM
Trying batch_size = 98... ✅ success
Trying batch_size = 99... ❌ OOM

✅ Optimal batch size: 98
```

In [ ]:
# Clear memory after benchmarking
release_all_gpu_memory(["model","model_pipeline","dataloader"])

### Run the pipeline on the whole dataset

In [ ]:
optimal_batch_size = 30

def build_pipeline(model_name, batch_size, ):
    model, tokenizer = load_mistral_4bit_model(model_name)
    model_pipeline = pipeline(
            "text-generation",
            model=model,
            batch_size=batch_size,
            tokenizer=tokenizer,
            #max_length=1024,  
            max_new_tokens=512,         # ✅ controls output size
            truncation=True,            # ✅ safely truncates long inputs
            return_full_text=False,     # ✅ excludes input from output
            do_sample=False
        )
    return model_pipeline

model_pipeline = build_pipeline(model_name, optimal_batch_size)

In [ ]:
# Use this cell to re-import a function in case you've updated it's code in summarizer.py
from importlib import reload
import summarizer
reload(summarizer)

# Now re-import manually
from summarizer import batch_generate_interpretations


In [ ]:

print("\n🧠 Running interpretations...")
tstp = datetime.now().strftime(r"%y.%m.%d-%H")

batch_generate_interpretations(dataset, model_pipeline, save_dir="output",
                               input_column="input", output_column="interpretation", 
                               batch_size=optimal_batch_size)
#print(result_df[["dream", "interpretation"]])


# Memory investigation

In [ ]:
from utils import globals_snapshot

tps = globals_snapshot()

q = """
select type, count(var) as cnt
from tps
group by type 
order by cnt desc 

"""
df = ps.sqldf(q)
df